In [2]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-07-14 16:40:53--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-07-14 16:40:53 (33.2 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [3]:
import minsearch

In [1]:
import json

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [11]:
from openai import OpenAI

In [12]:
client = OpenAI()

In [14]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

'It depends on the specific policies of the institution or organization offering the course. Here are a few steps you can take:\n\n1. **Check Enrollment Deadlines:** Review the course information, website, or syllabus to see if there are any specific enrollment deadlines.\n\n2. **Contact the Instructor or Administration:** Reach out to the course instructor or the admissions office to inquire if late enrollment is possible. Sometimes, exceptions can be made.\n\n3. **Review the Impact:** Consider how much of the course you have missed and if you can realistically catch up on the missed material.\n\n4. **Consult the Course Platform:** If the course is online, the platform may allow self-paced enrollment, which means you might still be able to join even after the official start date.\n\n5. **Understand the Policies:** Make sure you understand any implications of late enrollment, such as missed assignments or participation points.\n\nIt’s often worthwhile to ask, as some courses have flexi

In [15]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [16]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [18]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

'To run Kafka, here are some instructions for different setups based on the CONTEXT provided:\n\n1. **Java Kafka**:\n   - Navigate to the project directory.\n   - Run the Java class with the command:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Python Kafka**:\n   - Create a virtual environment and install the necessary packages:\n     ```bash\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - To activate the virtual environment in future sessions, run:\n     ```bash\n     source env/bin/activate\n     ```\n   - To deactivate the environment, use:\n     ```bash\n     deactivate\n     ```\n\nIf you encounter issues or need specific configurations, make sure you also have Docker images up and running as needed.'

In [20]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course after it has started. However, you need to be aware of the deadlines for submitting the final projects and ensure you don't leave everything until the last minute."

In [7]:
from elasticsearch import Elasticsearch

In [8]:
es_client = Elasticsearch('http://localhost:9200') 

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [6]:
from tqdm.auto import tqdm

In [10]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:25<00:00, 36.91it/s]


In [27]:
query = 'I just disovered the course. Can I still join it?'

In [29]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [30]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [31]:
rag(query)

"Yes, you can still join the course even if you just discovered it. You are eligible to submit the homework regardless of whether you registered or not. However, keep in mind that there will be deadlines for turning in the final projects, so make sure you don't leave everything for the last minute."